# Welcome to the first python workshop: Bit by Bit

Kindly organised by SBS Biohackathon Committee

Instructors for the day: Sundara and Min Qi

## Learning Objectives:

1. Understand Bits and Bitwise Operators
2. Build your own exact string matchig algorithm
3. Build your own inexact string matching algorithm
4. Apply Shift-Or algorithm to find genes of interest within queried genetic sequence

# UNDERSTANDING BITS

## What is Binary Code (Bits)?
- The bit is the most basic unit of information in computing and digital communications. 
- The bit represents a logical state with one of two possible values (0 or 1)
- Each digit position represents an increasing power of 2
- Functions: For flagging system; on/off switch; Storage etc.

## Practice! Convert the following from binary to decimal integers

- 10101
- 100011
- 11000

## Practice! Convert the following from decimal integers to binary

- 18
- 36
- 57

## Binary Conversion in python
- You can convert binary to integers using the int() function
- You can convert integers to binary in python using the bin() function
- python recognises binary using the 0b infront of the code

In [1]:
print(int(0b10101))
print(int(0b100011))
print(int(0b11000))

21
35
24


In [2]:
print(bin(18))
print(bin(36))
print(bin(57))

0b10010
0b100100
0b111001


# Bitwise Operators

## What are Bitwise Operators ?
- You can use bitwise operators to perform Boolean logic on individual bits.
- Similar to python Operators (+, -, % etc.) but specifically to compare binary numbers

## Bitwise AND 
- Denoted by: &
- Similar to _and_  Boolean Operator (i.e True and False = False) 
- For each pair of bits occupying the same position in the two numbers, it returns a 1 only when both bits are 1

In [3]:
a = 0b11001
b = 0b10101

print(bin(a))
print(bin(b))
print(bin(a&b))

0b11001
0b10101
0b10001


## Bitwise OR
- Denoted by: |
- Similar to _or_  Boolean Operator (i.e True or False = True; True or True = True; False or False = False) 
- For each pair of bits occupying the same position in the two numbers, it returns a 1 as long as one 1 is present in any of the numbers

In [4]:
c = 0b11001
d = 0b10101

print(bin(c))
print(bin(d))
print(bin(a|b))


0b11001
0b10101
0b11101


## Bitwise Left Shift
- Denoted by: <<
- Moves the bits of its first operand to the left
- Shifting a single bit to the left by one place doubles its value.

In [5]:
a = 0b10011

print(bin(a))
print(bin(a<<1))
print(bin(a<<2 ))

0b10011
0b100110
0b1001100


## Bitwise Right Shift
- Denoted by: >>
- Analogous to left shift; moves the bits of its first operand to the right
- Shifting a single bit to the right; dropping the rightmost bit


In [6]:
b = 0b1001101

print(bin(b))
print(bin(b>>1))
print(bin(b>>2))

0b1001101
0b100110
0b10011


## Summary 

- AND (__&__) : Sets each bit to1 only if both bits are 1
- OR (__|__) : Sets each bit to 1 if one of two bits is 1
- LEFT SHIFT (__<<__) : Shift left by pushing zeros in from the right and let the leftmost bits fall off
- RIGHT SHIFT (__>>__) : Shift right by pushing copies of the leftmost bit in from the left, and let the rightmost bits fall off

# Shift-Or Algorithm

## Step 1: Create a dictionary of bitmasks


A bitmask is a binary string with 0 and 1s representing the relative position of a single character within the pattern queried.


0 denotes a match of the letter in the position

1 denotes a match of the letter in the position


Every unique letter within the sequence must have a corresponding bitmask (its relative position in the pattern queried)


TO NOTE: 
    <reference> refers to text queried
    <query> refers to pattern queried within the text

In [7]:
#Function: create bitmask dictionary

def _generateAlphabet(reference, query): #defining function, reference is the seq and query is the pattern
	alphabet = list(set(reference)) #seperate the ref into iterable UNIQUE elements
	bitap_dict = {} #opening up a bitmask dictionary B[Tj] for each unique letter
	for letter in alphabet: #for every single unique element in the reference
		letterPositionInQuery = 0 #start off with 0
		for symbol in query:
			letterPositionInQuery = letterPositionInQuery << 1 #left shift to move specific letter across binary string (from left to right) to tally with the letter position in the query
			letterPositionInQuery |= int(letter != symbol) ## adds 1/0 to the end of binary string based on boolean value #check if the specified character matches corresponding character in the query in the specific position
		bitap_dict[letter] = letterPositionInQuery #repeat above block till last character in the query and update dictionary with the obtained bit mask of len (equals to query length)
	return bitap_dict #dictionary of <all possible letters (from seq/ref) in query/pattern> (key) and <letterPositionInQuery> (value)

In [8]:
#Try it yourself!

#reference: "ATTGCTCGATCG"
#pattern: "ATC"

for key,value in _generateAlphabet("ATTGCTCGATCG","ATC").items():
    print(key, value)

#Is this the desired output?
#Yes, but printing the value as is outputs the integer form of the binary string. Rerun the code with the use of the bin() function.

G 7
C 6
A 3
T 5


In [9]:
#Try it again

#reference: "ATTGCTCGATCG"
#pattern: "ATC"

for key,value in _generateAlphabet("ATTGCTCGATCG","ATC").items():
    print(key, bin(value))

#Note that any zeros occuring on the leftmost side will be truncated. 
    #e.g. 01 will be truncated to 1
    #therefore, bitmask of A is actually 0b011

G 0b111
C 0b110
A 0b11
T 0b101


In [10]:
from collections import namedtuple

placeholder = namedtuple('placeholder','query seq start end mismatch')

#placeholder is the name, and query seq start end mismatch is the associated value; better than dictionary since it is immutable and less writing is involved

## Step 2:Initialize bitarray (D)

A bitarray, D, is used like a placeholder to capture matches of each letter in the text to the queried pattern

*Must be the same length as the pattern

In [11]:
#Try it yourself! Try changing the queryLen

queryLen = 3
D = (2 << (queryLen - 1)) - 1

print(D)
print(bin(D))

#Why do we perform a left shift on 2?
#binary code of 2 (10) left shift (queryLen-1) spaces (the minus 1 to convert all binaries to 1 and reduce length of binary string by 1)
    #e.g. if queryLen is 4, 
    #2 has a binary string of 10 
    #2<<(4-1) --> 0b10 000
    # 0b10 000 -1 --> 0b1111 (binary string length reduce by 1 --> same length as queryLen)

7
0b111


## Step 3: Exact String Matching

Algorithm to find the instances of sequences that are identical to the pattern queried within the pattern.

In [12]:
#Function: Algorithm to look for exact matches

def bitapexactSearch(reference, query): 
	referenceLen = len(reference)
	queryLen = len(query)
	exact_placeholder = namedtuple('placeholder','query seq start end')

	alphabet = _generateAlphabet(reference, query)

	matrix = [] 
	emptyColumn = (2 << (queryLen - 1)) - 1
    
	matrix.append(emptyColumn)
	gRNAs = []

            
	for columnNum in range(1, referenceLen + 1):
		prevColumn = (matrix[columnNum - 1]) >> 1 #D'#right shift to move on to next letter 
		letterPattern = alphabet[reference[columnNum - 1]] #bitmask B[Tj] #Note that the letter index in reference (seq) will always be 1 less than columnNum (since columnNum starts with 1 to denote first letter in seq, which has index 0)
		curColumn = prevColumn | letterPattern
		matrix.append(curColumn)
        
		if (curColumn & 0x1) == 0: #0x1 represents hexadecimal 1 (binary string of 1) #in a right shift matching --> 0 in first position implies a match in first character (note that & 0x1 --> implies that curColumn must have a 0 in first posiiton to offset the 1 in 0x1)
			startPos = max(0, columnNum - queryLen) # taking in account Replace operation #ensures that the minimum length of a match is the query length
			endPos = min(columnNum, referenceLen) # taking in account Replace operation #finds the shortest possible instance of a match
			place = reference[startPos:endPos]
			temp = exact_placeholder(query, place, startPos, endPos)
			gRNAs.append(temp)
	return gRNAs

In [13]:
#Try it yourself!

reference  = 'ATCGATC'
string_search = 'ATC'
gRNAs = bitapexactSearch(reference, string_search)
for i, g in enumerate(gRNAs):
	print (g)

placeholder(query='ATC', seq='ATC', start=0, end=3)
placeholder(query='ATC', seq='ATC', start=4, end=7)


## Step 4: Inexact String Matching

Algorithm to find the instances of sequences that differ from the pattern by a specified number of characters within text.

Possible uses of inexact matching:
Looking for instances of mutated sequences of a specific gene of interest within a genome

In [14]:
#Function: Algorithm to look for exact and inexact matches

def bitapSearch(reference, query, mismatch = 1): #NOTE: mismatch refers to a mismatch of a single character between query and reference [number of mismatches defined refers to number of erroneous characters tolerated for a pattern to be found]
	referenceLen = len(reference) #sequence
	queryLen = len(query) #pattern

	alphabet = _generateAlphabet(reference, query)

	matrix = [] 
	emptyColumn = (2 << (queryLen - 1)) - 1
    
	underground = [emptyColumn for i in range(referenceLen + 1)] #underground is a list of 11111 (D) inital bitmasks for every letter in the reference (sequence)
	matrix.append(underground)
	gRNAs = []
	skip = []

	for k in range(1, mismatch + 2): 
		matrix.append([emptyColumn])
        #in the matrix:
            #index [0]: underground
            #index [1]: no mismatch (thats why the inexact matching only occurs for k>1, when k=1, the if clause does not run)
            #index [2]: 1 mismatch 
            #index [k]: k-1 mismatches
            #NOTE: k has a range of 1, mismatch+2 because k starts at 1, since index 0 of matrix is used for underground, and mismatch specified plus 2 because the range function does not run the max number stated
            #e.g. mismatch = 1, so k runs in range (1,3) --> k = 1, matrix[1]: stores no mismatch, k = 2, matrix[2]: stores 1 mismatch
        
        #When k=1, exact string search results are obtained  
		for columnNum in range(1, referenceLen + 1):
			prevColumn = (matrix[k][columnNum - 1]) >> 1 #D'#right shift to move on to next letter 
			letterPattern = alphabet[reference[columnNum - 1]] #bitmask B[Tj] #Note that the letter index in reference (seq) will always be 1 less than columnNum (since columnNum starts with 1 to denote first letter in seq, which has index 0)
			curColumn = prevColumn | letterPattern
            
            #INEXACT STRING SEARCH
			if k > 1:
				## Mismatch 
				curColumn = curColumn & (matrix[k - 1][columnNum - 1] >> 1)
				#(matrix[k - 1][columnNum - 1] >> 1) looks at the previous result for the same character in the same position, AND conditional on the current column and no mismatch D (matrix[1][columnNum-1])
                
                ##IMPORTANT NOTE: 
                    #OR: favours 1 (where 1,1  0,1  1,0 --> gives 1 and only 0,0 --> gives 0)
                    #OR function is more sensitive to mismatches --> hence used in exact string matching
                    #AND: favours 0 (where 0,0  0,1  1,0 --> gives 0 and only 1,1 --> gives 1)
                    #AND function tolerates mismatches --> hence used in inexact string matching
                    
			matrix[k].append(curColumn)

			if (curColumn & 0x1) == 0: #0x1 represents hexadecimal 1 (binary string of 1) #in a right shift matching --> 0 in first position implies a match in first character (note that & 0x1 --> implies that curColumn must have a 0 in first posiiton to offset the 1 in 0x1)
				startPos = max(0, columnNum - queryLen) # taking in account Replace operation #ensures that the minimum length of a match is the query length
				if startPos in skip: continue
				endPos = min(columnNum, referenceLen) # taking in account Replace operation #finds the shortest possible instance of a match
				place = reference[startPos:endPos]
				temp = placeholder(query, place, startPos, endPos, k - 1)
				gRNAs.append(temp)
				skip.append(startPos) #prevents repeated reporting of same strain
                #NOTE: inexact string search will also capture previous patterns where there are no mismatches and instances of fewer mismatches --> repeated reporting
			
	return gRNAs

In [15]:
#Try it yourself

reference  = 'GGGCNCTGCTGAGAATGNACTGAATATAAACTTGTGGTAGTTGGANGCTGGTGGCGTAGGCTTGTGGTTGTGGGANGCTGGTGGCGAAGAGTGCCTTGACGATACAGNCTANATTNCAGAATNCATTTTGTGGNACGAATATGATCCANACAATAGNAGGATTC'
string_search = 'TTGTGGTAGTTGGANGCTGGTGGCG' #pattern we are looking for
errors = 2 #allowing mismatch of 2; subsitution mutations to take place
gRNAs = bitapSearch(reference, string_search, errors)
for i, g in enumerate(gRNAs):
	print (g)

placeholder(query='TTGTGGTAGTTGGANGCTGGTGGCG', seq='TTGTGGTAGTTGGANGCTGGTGGCG', start=31, end=56, mismatch=0)
placeholder(query='TTGTGGTAGTTGGANGCTGGTGGCG', seq='TTGTGGTTGTGGGANGCTGGTGGCG', start=61, end=86, mismatch=2)
